In [1]:
import pandas as pd
import csv
import json
import aisuite as ai
client = ai.Client()

In [10]:
questions = []
with open('../data/questions_and_statements.json', 'r') as f:
    questions_data = json.load(f)
    questions = [entry["question"] for entry in questions_data]
    statements = [entry["statement"] for entry in questions_data]

# extract questions to run a test
test_questions = [questions[10], questions[11]]
test_statements = [statements[10], statements[11]]

# load the full summary dataframe
df = pd.read_csv('../data/movie_book_dataframe.csv')

# extract synopsis to run a test
test_summaries = df[df['Title'].isin(['Snowpiercer', 'Dracula'])]

# select models
models = ["openai:gpt-4o-mini"] #"anthropic:claude-3-5-sonnet-20240620"
test_questions
test_statements

['The theme of revenge is explored in this text.',
 'The theme of identity and self-exploration is explored in this text.']

In [ ]:
all_prompts = [
    {
        "title": row['Title'],
        "summary": row['Summary'],
        "binary": [
            {"role": "system", "content": 
             "You are a story analyzer. Your task is to analyze synopses regarding a specific question. You only answer with yes or no."}, 
            {"role": "user", "content": f"<Synopsis> {row['Summary']} </Synopsis> <Question>Answer the following question: {question}</Question>"}
        ],
        "statement": [
            {"role": "system", "content": 
             "You are a story analyzer. Your task is to analyze synopses regarding a given hypothesis. You indicate your agreement on a scale from 1 (strongly disagree) to 7 (strongly agree). You only answer with a number."}, 
            {"role": "user", "content": f"<Synopsis> {row['Summary']} </Synopsis> <Question>How much do you agree with the following statement (1-7): {statement}</Question>"}
        ]
    }
    for _, row in test_summaries.iterrows()
    for question, statement in zip(test_questions, test_statements)
]

In [22]:
for model in models:
    for i in range(len(all_prompts)):
        question_prompt = all_prompts[i]["binary"]
        statement_prompt = all_prompts[i]["statement"]

        binary_response = client.chat.completions.create(
        model=model,
        messages=question_prompt,
        temperature=0)

        likert_response = client.chat.completions.create(
        model=model,
        messages=statement_prompt,
        temperature=0)

        all_prompts[i][f"binary_response_{model}"] = binary_response.choices[0].message.content
        all_prompts[i][f"likert_response_{model}"] = likert_response.choices[0].message.content

        #update json file
        with open('../data/ai_responses.json', 'w') as f:
            json.dump(all_prompts, f, indent=4)



In [21]:
len(all_prompts)

4

In [ ]:
[entry["binary_response"] for entry in all_prompts]
all_prompts

[{'title': 'Dracula',
  'summary': "Dracula is an epistolary novel, meaning that is composed from letters, journal and diary entries, telegrams, and newspaper clippings. Jonathan Harker, Mina Murray , and Dr. Seward write the largest contributions to the novelalthough the writings of Lucy Westenra and Abraham Van Helsing constitute some key parts of the book. The novel is meant to have a slightly journalistic feel, as it is a harrowing account supposedly written by the people who witnessed the book's events. A young Englishman named Jonathan Harker travels through Transylvania on a business trip. He is there to aid Count Dracula, a Transylvanian nobleman, in buying an English estate. His journey into the remote Eastern European landscape is fearsome, although initially he is charmed by the Count's generosity and intelligence. Gradually, he comes to realize that he is a prisoner in Dracula's castle, and that the Count is a demonic being who plans to prey on the teeming masses of London.